##  Problema 1: Reubicacion de ejecutables

Todos los SO tienen un modulo que es el cargador o loader, este modulo es el encargado de que cuando se quiere lanzar un proceso nuevo, lo que tiene que hacer es levantar los datos del proceso, ver cuanto lugar de memoria necesita, levantar el programa de disco a esa zona de memoria y luego le transferira el control. En definitiva el loader necesita alocar la memoria y subir el codigo. Evidentmente el programa necesita ser modificado para que corra en una posicion de memoria que no es la posicion en la cual se compilo. Obviamente el loader tiene que modificar el programa, y como hace? este proceso de modificar el programa para poder ejecutar correctamente se lo conoce como static relocation. Queda claro que es imposible deducir cuales son esas posiciones, tienen que ser parte del ejecutable. Este proceso de que cada tarea que levanto la tengo que subir a RAM y la tengo que corregir es perdida de tiempo. Asi y todo se manejo asi durante mucho tiempo.

## Problema 2: Proteccion entre tareas

El hecho de usar la relocacion estatica nos resolvio el problema de poner el mismo codigo a ejecutar en distintas posiciones (se necesita esto para multiprocesamiento). Resolvimos con la relocacion estatica como hacer para que los codigos puedan ejecutar en paralelo, asi y todo no hemos resuelto otras cosas. Por ahora no tenemos una forma de protejer que alguien no acceda a una direccion de memoria que no debe. Tenemos que ver que mecanismos vamos a tener para implementar la proteccion de memoria. Yo podria modificar otra tarea, si puedo apuntar a la memoria de otra tarea y no hay nada que me detenga entonces podria modificarla.

## Primer solucion al tema de seguridad: Segmentado

Es una de las formas de manejo de memoria mas comunes y clave. Si queremos que esto funcione rapido involucra hardware. El segmentado es dotar al procesador de un registro mas o juego de registros. Cada vez que se tiene una direccion para sacar al bus de direcciones, en realidad, hay un sumador por hardware y la posicion que se va a buscar no es la posicion que pidio el programa sino que es la posicion mas el registro. Agregamos ese registro, hacemos la reubicacion del programa, ya no necesito modificar el programa, lo unico que tiene que hacer el SO es decir donde lo va a correr y cargar el registro. Ahora, ya que agregamos el sumador en hardware porque no agregamos tambien un comparador. Cada vez que el programa va a acceder a la memoria le sumo la base pero verifico que no me pase del limite. Ahora el hardware me da la proteccion. El SO pasa a tener un rol de supervision, configura la base, configura el limite y deja que el programa corra. Ahora tenemos una base y un limite, ya resolvimos el problema de la relocacion y de la seguridad. Nos queda un tema para terminar de decir que esto es seguro, que pasa si la tarea cambia el registro de segmentacion? claramente el cambio de registro tiene que ser algo protegido, la solucion mas facil seria, la instruccion que carga un registro de segmentacion es exlcusiva de ring 0, esa es una alternativa, otra alternativa mas usada es, cuando alguien quiere cargar un registro de segmentacion voy a hacer una serie de chequeos, si estas cambiando un registro de segmentacion por otro valido te voy a dejar y sino no. Como era en el 8086, no se podian hacer todas estas cosas, tenian los registros de segemtnacion y servian para reubicar programas en lo que se llamaba dinamic realocation pero no tenia limite. No habia proteccion contra el cambio de registros, era como una especie de multitarea colaborativa. Si cumplia la funcion de la relocacion dinamica. Como funcionaba? solamente con sumador. Permitia acceder maximo a 20 bits. La conclusion es que resolvia la relocacion pero nada de seguridad. Cuando sacan 80286 este empieza a trabajar en modo protegido. Fue medio un fracaso pero se extendio el concepto de segmento y se introducen las tablas de segmento por primera vez, que se usan hasta hoy en dia. En particular define dos tablas, escencialmente, el registro de segmentacion deja de ser la base o el offset sino que un indice en una tabla, en esa tabla el SO puso la base, el limite y las prioridades. Tenemos nuestros segmentos con base y limite, los tenemos en tablas en zona de memoria. Cada vez que se va a cargar un registro de segmentacion, una carga exitosa termina en la eleccion de una de las dos tablas de segmentacion, la eleccion de un indice, la carga de todos los valores de esa entrada en la tabla a una parte oculta del regsitro de segementacion. Que chequeo se hace? el proceso desde el procesador soporta distintos niveles de privilegio, el SO ejecuta en ring 0, pero cuando lanza unaa tarea la corre como ring 3. correr como ring 3 es que el segmento que le doy de codigo que son 16 bits, en los dos primeros le pone 1 1, ese segmento corre con prioridad 3. Entonces, cuando se quiere cargar un registro de segementacion, la instruccion de carga de registro no es una instruccion prohibida para una tarea, las tareas pueden cargar el registro de segmentacion. 

## Memory Swapping

El loader primero levanta el programa, ve que tamanio tiene y se fija donde lo pone en memoria. Que pasa si el programa no entra en ningun agujero de memoria. En cualquiera de los escenarios vistos, si se desea ejecutar una nueva tarea y no hay memoria, el SO puede copiar la imagen en memoria de algun proceso y liberar el lugar necesario para correrla. Cuando no habia apoyo del hardware, el SO debia elegir arbitrariamnete una tarea para bajarla. Una vez elegida quedaba completamente inactiva hasta que se decidiera volver a subirla a RAM. Esto era asi aunque la tarea tubiera un gran bloque de memoria que no estaba en uso y en el cual entraria el proceso nuevo que se quiere correr. Esto se conoce como full-swap. En momentos de la historia en que la necesidad de memoria en los programas creció más rápidamente que la disponible en los ordenadores, los programadores recurrieron el concepto de Overlays. Esta técnico consistía en dividir el código en bloques elegidos con detalla de forma tal que no todo el programa tuviera que estar en memoria a la vez. Lo que se veía del lado del usuario, por ejemplo al ingresar al autocorrector de un procesador de texto, era una pequeña demora que coincidía con la luz de acceso a disco. Lo que estaba pasando es que el SO estaba bajando a disco, a pedido del mismo programa digamos el módulo de definición de gráficos para subir el del 
autocorrector.
